#### MP based asynchronous testing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import multiprocessing as mp
import TestPool_Unit
from shutil import copyfile
import numpy as np

In [3]:
from collections import defaultdict

In [4]:
def Evaluate(result_arr):
    print('Files Processed: ', len(result_arr))
    recalls = []
    recalls_of_word = []
    precisions = []
    precisions_of_words = []
    fully_Correct_l = 0
    fully_Correct_w = 0
    for entry in result_arr:
        (word_match, lemma_match, n_dcsWords, n_output_nodes) = entry
        recalls.append(lemma_match/n_dcsWords)
        recalls_of_word.append(word_match/n_dcsWords)

        precisions.append(lemma_match/n_output_nodes)
        precisions_of_words.append(word_match/n_output_nodes)
        if lemma_match == n_dcsWords:
            fully_Correct_l += 1
        if word_match == n_dcsWords:
            fully_Correct_w += 1
    print('Avg. Micro Recall of Lemmas: {}'.format(np.mean(np.array(recalls))))
    print('Avg. Micro Recall of Words: {}'.format(np.mean(np.array(recalls_of_word))))
    print('Avg. Micro Precision of Lemmas: {}'.format(np.mean(np.array(precisions))))
    print('Avg. Micro Precision of Words: {}'.format(np.mean(np.array(precisions_of_words))))
    rl = np.mean(np.array(recalls))
    pl = np.mean(np.array(precisions))
    print('F-Score of Lemmas: ', (2*pl*rl)/(pl+rl))
    print('Fully Correct Lemmawise: {}'.format(fully_Correct_l/len(recalls_of_word)))
    print('Fully Correct Wordwise: {}'.format(fully_Correct_w/len(recalls_of_word)))
    print('[{:0.2f}, {:0.2f}, {:0.2f}, {:0.2f}, {:0.2f}, {:0.2f}, {:0.2f}]'.format(100*np.mean(np.array(recalls)), 100*np.mean(np.array(recalls_of_word)), 100*np.mean(np.array(precisions)), \
           100*np.mean(np.array(precisions_of_words)), 100*(2*pl*rl)/(pl+rl), 100*fully_Correct_l/len(recalls_of_word),\
           100*fully_Correct_w/len(recalls_of_word)))

In [5]:
modelFile = 'outputs/train_t2756013734745/nnet_e1_i400.p' #PM3
_dump = False
if _dump:
    _outFile = 'outputs/dump_predictions/PM3_NLoss'
else:
    _outFile = None

# Backup the model file
copyfile(modelFile, modelFile + '.bk')

# Create Queue, Result array
queue = mp.Queue()
result_arr = []

# Start 6 workers - 8 slows down the pc
proc_count = 4
procs = [None]*proc_count
for i in range(proc_count):
    vpid = i
    procs[i] = mp.Process(target = TestPool_Unit.pooled_Test, args = \
                          (modelFile, vpid, queue, '../NewData/skt_dcs_DS.bz2_1L_pmi_mir_heldout_again/', int(9600/proc_count), _dump, _outFile))
# Start Processes
for i in range(proc_count):
    procs[i].start()

Child process with vpid:0, pid:22127 started.
Child process with vpid:1, pid:22128 started.
Child process with vpid:2, pid:22131 started.
Child process with vpid:3, pid:22134 started.
Keep Prob = 0.6, Dropout = 0.4
Keep Prob = 0.6, Dropout = 0.4
Keep Prob = 0.6, Dropout = 0.4
Keep Prob = 0.6, Dropout = 0.4
vpid:2: Range is 4800 -> 7200 / 9577
vpid:0: Range is 0 -> 2400 / 9577
vpid:1: Range is 2400 -> 4800 / 9577
vpid:3: Range is 7200 -> 9600 / 9577


In [19]:
# Fetch partial results
while not queue.empty():
    result_arr.append(queue.get())
# Evaluate results till now
Evaluate(result_arr)

Files Processed:  5985
Avg. Micro Recall of Lemmas: 0.8412894676055079
Avg. Micro Recall of Words: 0.6757667871931413
Avg. Micro Precision of Lemmas: 0.7936086846797896
Avg. Micro Precision of Words: 0.6385993466612693
F-Score of Lemmas:  0.816753786024
Fully Correct Lemmawise: 0.38395989974937345
Fully Correct Wordwise: 0.1241436925647452
[84.13, 67.58, 79.36, 63.86, 81.68, 38.40, 12.41]


In [7]:
# Check status
for i in range(proc_count):
    p = procs[i]
    print('Process with\t vpid: {}\t ->\t pid: {}\t ->\t running status: {}'.format(i, p.pid, p.is_alive()))

Process with	 vpid: 0	 ->	 pid: 22127	 ->	 running status: True
Process with	 vpid: 1	 ->	 pid: 22128	 ->	 running status: True
Process with	 vpid: 2	 ->	 pid: 22131	 ->	 running status: True
Process with	 vpid: 3	 ->	 pid: 22134	 ->	 running status: True


In [22]:
# Properly Join
for i in range(proc_count):
    procs[i].join()

In [20]:
# Force Terminate
for p in procs:
    p.terminate()

In [24]:
np.mean([a[2] for a in result_arr])

6.7729026036644164

In [12]:
result_arr2 = [list(x) for x in result_arr]

recalls = defaultdict(list)
for i in range(len(result_arr2)):
    result_arr2[i][0] /= result_arr2[i][2]
    result_arr2[i][1] /= result_arr2[i][2]
    recalls[result_arr2[i][2]].append(result_arr2[i][1])

for c in sorted(recalls.keys()):
    print('C: {}, Samples: {}, Mean: {}'.format(c, len(recalls[c]), np.mean(recalls[c])))

C: 1, Samples: 146, Mean: 0.7328767123287672
C: 2, Samples: 313, Mean: 0.8482428115015974
C: 3, Samples: 216, Mean: 0.875
C: 4, Samples: 191, Mean: 0.8756544502617801
C: 5, Samples: 478, Mean: 0.9041841004184101
C: 6, Samples: 1479, Mean: 0.9195402298850575
C: 7, Samples: 1976, Mean: 0.9254626951995374
C: 8, Samples: 1403, Mean: 0.9167854597291518
C: 9, Samples: 519, Mean: 0.9096553200599445
C: 10, Samples: 152, Mean: 0.9151315789473685
C: 11, Samples: 69, Mean: 0.9064558629776023
C: 12, Samples: 57, Mean: 0.8947368421052633
C: 13, Samples: 20, Mean: 0.8807692307692309
C: 14, Samples: 11, Mean: 0.9220779220779222
C: 15, Samples: 13, Mean: 0.8871794871794874
C: 16, Samples: 6, Mean: 0.8958333333333334
C: 17, Samples: 9, Mean: 0.8888888888888888
C: 18, Samples: 7, Mean: 0.9285714285714286
C: 19, Samples: 4, Mean: 0.8552631578947368
C: 20, Samples: 1, Mean: 0.95
C: 21, Samples: 4, Mean: 0.9523809523809523
C: 22, Samples: 1, Mean: 0.6363636363636364
C: 23, Samples: 2, Mean: 0.8913043478260